In [3]:
!pip install -U openai-whisper

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 25.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 22.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 16.9 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 22.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801360 sha256=1ed5e5fbeccecc4836bae

In [7]:
!pip install pytubefix

Defaulting to user installation because normal site-packages is not writeable


## Download audio from YouTube video

In [8]:
import pytubefix as pt

yt = pt.YouTube("https://www.youtube.com/watch?v=eK0GXrZpuqs")
stream = yt.streams.filter(only_audio=True)[0]
stream.download(filename="audio.mp3")

'/home/guilalire/Personal/Discover-France-LLM/notebooks/audio.mp3'

## Transcribe audio using OpenAI Whisper

In [9]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("audio.mp3")
text = result['text']
language = result['language']

text

" 17 millions c'est le nombre de personnes qui ont regardé la cérémonie de clôture des jeux olympiques de Paris, ce dimanche soir au stade de France. Une cérémonie qui signe donc la fin de ces géos 2024, qui apparaissent comme une vraie réussite au lieu des Français évidemment, mais aussi du monde entier. Alors comment l'expliquer ? Salut c'est Blanche, j'espère que vous allez bien, c'est le sujet à la une de ces actus du jour. La première réussite de ces jeux, ce sont les audiences télé, mais aussi la vente de billets pour les épreuves. La couteille audience, 58,4 millions de personnes, suivent les jeux olympiques sur les antennes de France, les visions, selon le groupe audiovisuel. Le pic d'audience a été atteint sur France 2, le 2 ou dernier, lors de la 4e médaille d'or du nageur Léon Marchand, avec 14,5 millions de télé-spectateurs. Côté billeterie, 9,5 millions de billets ont été vendus, soit plus que le record des 8,3 millions de billets vendus au jeu d'Atlanta en 1986. La deuxiè

## Translate text to English

In [10]:
import re

def split_text_into_chunks(text, chunk_size):
    sentences = re.split(r'(?<=[.!?])\s+', text)  # Split text by sentence-ending punctuation followed by a space
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        # If adding the next sentence exceeds the chunk size, save the current chunk and start a new one
        if len(current_chunk) + len(sentence) + 1 > chunk_size:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
        else:
            current_chunk += " " + sentence

    # Append the last chunk
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [11]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = language

CHAR_LIMIT = 800
chunks = split_text_into_chunks(text, CHAR_LIMIT)

translated_text = ""

for chunk in chunks:
    encoded_hi = tokenizer(chunk, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_hi,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
    )
    translated_text += tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0] + " "
    
translated_text

"17 million is the number of people who have watched the closing ceremony of the Olympic Games in Paris, this Sunday evening in the stadium of France. A ceremony that thus signs the end of these ghosts 2024, which appear as a real success in place of the French obviously, but also of the whole world. So how do you explain it? Welcome to Blanche, I hope you're going well, it's the subject of one of these actus of the day. The first success of these games, are the televised audiences, but also the sale of tickets for the events. The eye-catcher audience, 58.4 million people, watch the Olympic Games on the antennas of France, the visions, according to the audiovisual group. The peak of the audience was reached on France 2, the 2nd or last, during the 4th gold medal of swimmer Léon Marchand, with 14.5 million television viewers. The ticket office, 9.5 million tickets were sold, or more than the record of 8.3 million tickets sold at the 1986 Atlantic Games in 1986. The second success, I don